<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/snowball/Parseo_de_Discursos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import numpy as np


import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d leonardocaravaggio/scraps-pdfs

mkdir: cannot create directory ‘/root/.kaggle’: File exists
Mounted at /content/drive
... resuming from 752167433 bytes (198962348 bytes left) ...
 99% 898M/907M [00:02<00:00, 102MB/s] 
100% 907M/907M [00:02<00:00, 85.2MB/s]


In [ ]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [ ]:
# Cargamos la base
base = pd.read_csv('/content/Scraps PDFs.csv')

In [ ]:
import pandas as pd
import re
from datetime import datetime

month_mapping = {
    "enero": 1,
    "febrero": 2,
    "marzo": 3,
    "abril": 4,
    "mayo": 5,
    "junio": 6,
    "julio": 7,
    "agosto": 8,
    "septiembre": 9,
    "octubre": 10,
    "noviembre": 11,
    "diciembre": 12,
}


pd.options.mode.chained_assignment = None
def generar(l):
  df=pd.DataFrame()
  lineas = l[2].strip().split("\n")

  coincidencias_nombre_cargo = []
  coincidencias_discurso = []
  discurso_actual = ""

  for linea in lineas:
      try:
        if linea[0].isupper() and (":" in linea or "(" in linea):
          coincidencias_nombre_cargo.append(linea.split(":")[0].strip())
          coincidencias_discurso.append(discurso_actual)
          discurso_actual = linea.strip()
        else:
          discurso_actual += " " + linea.strip()
      except: discurso_actual = " "

  # Agregar la última coincidencia de discurso
  coincidencias_discurso.append(discurso_actual)

  if coincidencias_nombre_cargo!=[] and linea!="(Corrección de errores)":
    # Verificar si hay la misma cantidad de coincidencias para nombre/cargo y discursos
    if len(coincidencias_nombre_cargo) == len(coincidencias_discurso[1:]):

        # Crear el DataFrame de pandas
        data = {
            "Nombre/Cargo": coincidencias_nombre_cargo,
            "Discurso": coincidencias_discurso[1:],
        }

        df = pd.DataFrame(data)
        df["Discurso"] = df["Discurso"].str.strip()



    else:
        print("Error: no coincide la cantidad de coincidencias para nombre/cargo y discursos.")

    df = df[df['Nombre/Cargo'].str.contains(r'[A-Z]{2,}')]
    discurso=df.apply(lambda row: re.sub(re.escape(row['Nombre/Cargo']) + r':\s*', '', row['Discurso']), axis=1)
    df['Discurso'] = discurso
    df['Cargo'] = df['Nombre/Cargo'].apply(lambda x: ' '.join(re.findall(r'\b[A-Z][A-ZÁÉÍÓÚÜÑ-]+\b', x)))
    df['Nombre'] = df['Nombre/Cargo'].apply(lambda x: re.findall(r'', x)[0] if re.findall(r'', x) else '')
    df['Link']=l[1]
    df['Legislatura']=l[1][27:29]
    df['tipo']= l[2][:re.search('\n', l[2]).span()[0]]

    ordenes=[]

    elementos_a_eliminar = ['\n', ' .', '.', ' \n','  \n', ' L.', '  SENADO D.', ' DIARIO DE SESIONES DEL SENADO\nJUNTA PREPARATORIA Y SESIÓN CONSTITUTIVA\n',
                              'DIARIO DE SESIONES DEL SENADO\nJUNTA PREPARATORIA Y SESIÓN CONSTITUTIVA\n', 'GPMX .', 'D.', 'SUMARIO', 'DIARIO DE SESIONES DEL SENADO', 'GPMX.',
                              'XUNTA DE GALICIA.', 'ESCUDERO MÁRQUEZ', 'NIA GARAU JUAN', 'DE DICIEMBRE.', 'DE', 'A', 'DE ABRIL.', 'GPP .', 'OCDE.', 'GPS.',
                              'PEDRO MANUEL HERNÁNDEZ LÓPEZ.', 'CELEBRADA LOS', 'QUE FALTAN.', 'EN TODO EL ESTADO.', 'GPCIU', 'DESARROLLO E', 'GPEPC .', ' DIARIO DE SESIONES DEL SENADO',
                            ' A ', ' N ÚM.', ' M.']

    for discurso in df['Discurso']:

      try:
        anterior=l[2].split(discurso)
      except:
        anterior=l[2]

      matches=re.findall(r'[A-ZÁÉÍÓÚÑ\s]+[\n.]', anterior[0], re.UNICODE)

      orden = [elemento for elemento in matches if elemento not in elementos_a_eliminar]

      if matches:
          if len(orden) > 0:
              ordenes.append(orden[-1].replace('\n', ''))
          else:
              ordenes.append('')
      else:
          ordenes.append('')

    df['Orden']=ordenes

    expresion_regular = r"(\d+)\s+de\s+([a-zA-Z]+)\s+de\s+(\d+)"
    fecha=re.search(expresion_regular, str(l).replace('’',''))[0]
    df['fecha']=fecha

    if fecha.find('de')-3<0:
      if fecha[:fecha.find('de')-1]=='':
        day=1
      else:
        day=int(fecha[:fecha.find('de')-1])
    else:
      day=int(fecha[fecha.find('de')-3:fecha.find('de')-1])


    m=month_mapping.get(fecha[fecha.find('de')+3:fecha[fecha.find('de')+1:].find('de')+fecha.find('de')].lower())
    if m is None:
      month=1
    else:
      month=int(m)

    if fecha[-4:]=='':
      year=int(1000)
    else:
      year=int(fecha[-4:])
    df['fecha_formato']=datetime(year, month, day)

  if df.empty:
    df['Nombre/Cargo']=None
    df['Discurso']=None
    df['Cargo']=None
    df['Nombre']=None
    df['Link']=None
    df['Legislatura']=None
    df['tipo']=None
    df['fecha']=None
    df['fecha_formato']=None

  return df

In [ ]:
base2=pd.DataFrame(columns=['Nombre/Cargo', 'Discurso', 'Orden','Cargo', 'Nombre', 'Link', 'Legislatura', 'tipo', 'fecha', 'fecha_formato'])

In [ ]:
for row in base.values:
  base2 = pd.concat([base2, generar(row)], axis=0)

In [ ]:
base2=base2.reset_index(drop=True)

In [ ]:
base2['fecha_formato'] = pd.to_datetime(base2['fecha_formato'], errors='coerce')

In [ ]:
grupos=pd.read_xml('/content/openData2023731165138.xml')

In [ ]:
base2['nombre']=""
base2['apellidos']=""
base2['procedLiteral']=""
base2['procedLugar']=""
base2['grupoNombre']=""


lista = []

for linea in base2.values:

    nombres = grupos.loc[grupos['apellidos'] == linea[3].upper()].nombre.tolist()

    if next(iter(nombres or []), None)==None:
      nombres = grupos.loc[grupos['apellidos'] == linea[2].upper()].nombre.tolist()
      linea[9]=next(iter(nombres or []), None)

      apellidos = grupos.loc[grupos['apellidos'] == linea[2].upper()].apellidos.tolist()
      linea[10]=next(iter(apellidos or []), None)

      procedLiteral	 = grupos.loc[grupos['apellidos'] == linea[2].upper()].procedLiteral.tolist()
      linea[11]=next(iter(procedLiteral or []), None)

      procedLugar	 = grupos.loc[grupos['apellidos'] == linea[2].upper()].procedLugar.tolist()
      linea[12]=next(iter(procedLugar or []), None)

      grupoNombre	= grupos.loc[grupos['apellidos'] == linea[2].upper()].grupoNombre.tolist()
      linea[13]=next(iter(grupoNombre or []), None)

    else:

      linea[9]=next(iter(nombres or []), None)

      apellidos = grupos.loc[grupos['apellidos'] == linea[3].upper()].apellidos.tolist()
      linea[10]=next(iter(apellidos or []), None)

      procedLiteral	 = grupos.loc[grupos['apellidos'] == linea[3].upper()].procedLiteral.tolist()
      linea[11]=next(iter(procedLiteral or []), None)

      procedLugar	 = grupos.loc[grupos['apellidos'] == linea[3].upper()].procedLugar.tolist()
      linea[12]=next(iter(procedLugar or []), None)

      grupoNombre	= grupos.loc[grupos['apellidos'] == linea[3].upper()].grupoNombre.tolist()
      linea[13]=next(iter(grupoNombre or []), None)

    lista.append(linea)

In [ ]:
base2=pd.DataFrame(lista, columns=base2.columns)

In [ ]:
column_name = 'apellidos'
null_count = base2[column_name].isnull().sum()

In [ ]:
null_count

In [ ]:
base2[base2[column_name].isnull()].Cargo.value_counts()[:20]

In [ ]:
from google.colab import files
base2.to_csv('base_parseada.csv', escapechar='\\')
files.download('base_parseada.csv')

In [ ]:
base2['Orden'][498022]